In [1]:
#Title: Week 8: Term Project Milestone 2: Data Preparation
#Author: Brett Werner
#Date: 02 Nov 2025
#Created By: Sathya Raj Eswaran
#Description: Term Project Milestone 2: Data Preparation
#===========================================

In [22]:
# Importing Libraries
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
warnings.filterwarnings('ignore')

In [24]:
# Load the dataset
df = pd.read_csv("data\\E-commerce Customer Behavior Dataset.csv")
# Display the first 5 rows to visually inspect the data.
print(f"Successfully loaded")
print("\nFirst 5 rows of the DataFrame:")
print(df.head())

Successfully loaded

First 5 rows of the DataFrame:
   Customer ID  Gender  Age           City Membership Type  Total Spend  \
0          101  Female   29       New York            Gold      1120.20   
1          102    Male   34    Los Angeles          Silver       780.50   
2          103  Female   43        Chicago          Bronze       510.75   
3          104    Male   30  San Francisco            Gold      1480.30   
4          105    Male   27          Miami          Silver       720.40   

   Items Purchased  Average Rating  Discount Applied  \
0               14             4.6              True   
1               11             4.1             False   
2                9             3.4              True   
3               19             4.7             False   
4               13             4.0              True   

   Days Since Last Purchase Satisfaction Level  
0                        25          Satisfied  
1                        18            Neutral  
2            

In [26]:
# 1. Data Cleaning
# Drop rows with missing 'Satisfaction Level'

print(f"Original Row Count: {len(df)}")
df_temp = df.dropna(subset=['Satisfaction Level']).copy()

print(f"Final Row Count (after dropping 2 NaNs): {len(df_temp)}")
print("\nFinal Data Info:")


Original Row Count: 350
Final Row Count (after dropping 2 NaNs): 348

Final Data Info:


In [28]:
# Identify the feature to drop
features_to_drop = ['Customer ID']

# Drop the non-useful feature
df_dropped = df_temp.drop(columns=features_to_drop)

In [30]:
# Display the first 5 rows of the new DataFrame to confirm the drop
print(f"Features dropped: {'Customer ID'}")
print("\nFirst 5 rows of the DataFrame after dropping 'Customer ID':")
print(df_dropped.head().to_markdown(index=False, numalign="left", stralign="left"))

print("\nShape of the DataFrame before dropping:", df.shape)
print("Shape of the DataFrame after dropping:", df_dropped.shape)

Features dropped: Customer ID

First 5 rows of the DataFrame after dropping 'Customer ID':
| Gender   | Age   | City          | Membership Type   | Total Spend   | Items Purchased   | Average Rating   | Discount Applied   | Days Since Last Purchase   | Satisfaction Level   |
|:---------|:------|:--------------|:------------------|:--------------|:------------------|:-----------------|:-------------------|:---------------------------|:---------------------|
| Female   | 29    | New York      | Gold              | 1120.2        | 14                | 4.6              | True               | 25                         | Satisfied            |
| Male     | 34    | Los Angeles   | Silver            | 780.5         | 11                | 4.1              | False              | 18                         | Neutral              |
| Female   | 43    | Chicago       | Bronze            | 510.75        | 9                 | 3.4              | True               | 42                         | Unsatis

Explanation for Dropping the Feature
Feature	Reason for Dropping
Customer ID	Non-Predictive Unique Identifier: This column assigns a unique number to each row/customer. It has no inherent predictive value for a machine learning model because its value is unique and arbitrary; it does not help the model generalize patterns to new, unseen customers. Using it would lead to overfitting and poor performance on new data.

In [37]:
# 2. Convert 'Discount Applied' from boolean to integer (0 or 1) for modeling
df_dropped['Discount Applied'] = df_dropped['Discount Applied'].astype(int)
print(df_dropped.info())
print("\nFirst 5 rows of the Final Cleaned DataFrame:")
print(df_dropped.head().to_markdown(index=False, numalign="left", stralign="left"))

<class 'pandas.core.frame.DataFrame'>
Index: 348 entries, 0 to 349
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Gender                    348 non-null    object 
 1   Age                       348 non-null    int64  
 2   City                      348 non-null    object 
 3   Membership Type           348 non-null    object 
 4   Total Spend               348 non-null    float64
 5   Items Purchased           348 non-null    int64  
 6   Average Rating            348 non-null    float64
 7   Discount Applied          348 non-null    int32  
 8   Days Since Last Purchase  348 non-null    int64  
 9   Satisfaction Level        348 non-null    object 
dtypes: float64(2), int32(1), int64(3), object(4)
memory usage: 28.5+ KB
None

First 5 rows of the Final Cleaned DataFrame:
| Gender   | Age   | City          | Membership Type   | Total Spend   | Items Purchased   | Average Rating   | Disc

Handled Missing Target Variable: The 2 rows with missing values in the crucial target column, Satisfaction Level, were dropped using dropna().. This is a safe and common practice when the number of missing values is minimal, ensuring a clean target for supervised learning. 2. Feature Transformation: The boolean feature Discount Applied was converted to an integer (1 for True, 0 for False) to ensure it is in a numerical format suitable for most machine learning models.

In [40]:
# --- Feature Transformation (Encoding) ---

# 5. One-Hot Encoding for nominal features: 'Gender' and 'City'
# 'drop_first=True' is used to avoid multicollinearity.
df_F = pd.get_dummies(df_dropped, columns=['Gender', 'City'], drop_first=True)

# 6. Ordinal Encoding for 'Membership Type' (assuming a logical order: Bronze < Silver < Gold)
membership_order = {'Bronze': 1, 'Silver': 2, 'Gold': 3}
df_F['Membership Type Encoded'] = df_F['Membership Type'].map(membership_order)
df_F = df_F.drop(columns=['Membership Type'])

# 7. Label Encoding for the target variable 'Satisfaction Level'
# (Assigning levels: 0=Unsatisfied, 1=Neutral, 2=Satisfied)
satisfaction_order = {'Unsatisfied': 0, 'Neutral': 1, 'Satisfied': 2}
df_F['Satisfaction Level Encoded'] = df_F['Satisfaction Level'].map(satisfaction_order)
df_F = df_F.drop(columns=['Satisfaction Level'])

The original categorical columns were transformed as follows:

1. Nominal Feature Encoding (One-Hot Encoding)
Nominal features are those without a clear intrinsic order. They were converted into binary (0 or 1) columns.

Gender and City were One-Hot Encoded. The drop_first=True option was used to avoid multicollinearity (the Dummy Variable Trap).

Gender was transformed into Gender_Male (Female is represented by Gender_Male=0).

City was transformed into five new city columns (e.g., City_Houston, City_New York), with City_Chicago serving as the baseline (all city columns set to 0).

2. Ordinal Feature Encoding
Ordinal features have a meaningful, ranked order. They were mapped to integer values reflecting that order.

Membership Type was Ordinal Encoded based on hierarchy:

Bronze: 1

Silver: 2

Gold: 3

The original Membership Type column was dropped and replaced with Membership Type Encoded.

3. Target Variable Encoding (Label Encoding)
The target variable for classification was converted to integers.

Satisfaction Level was Label Encoded to define the classes for the model:

Unsatisfied: 0

Neutral: 1

Satisfied: 2

The original Satisfaction Level column was dropped and replaced with Satisfaction Level Encoded.

This code transformed the categorical data into numerical data suitable for machine learning:

Gender and City were converted using One-Hot Encoding.

Membership Type was converted using Ordinal Encoding (1, 2, 3).

Satisfaction Level (the target) was converted using Label Encoding (0, 1, 2).

In [46]:
# Convert 'Discount Applied' from boolean (True/False) to integer (1/0)
df_F['Discount Applied'] = df_F['Discount Applied'].astype(int)

# 3. Engineer New Useful Features (RFM-derived metrics)
# Average Item Price: Captures the customer's value segment (how expensive their items are)
df_F['Average_Item_Price'] = df_F['Total Spend'] / (df_F['Items Purchased'])

# Engagement Score: Measures frequency/volume relative to recency
df_F['Engagement_Score'] = df_F['Items Purchased'] / (df_F['Days Since Last Purchase'] )

# Recency Value Ratio: Measures monetary value relative to recency (high-value, active customer)
df_F['Recency_Value_Ratio'] = df_F['Total Spend'] / (df_F['Days Since Last Purchase'] )

New, highly useful features were engineered from the existing raw data, which often capture more meaningful customer behavior metrics than the original features alone. These new features are based on common e-commerce and RFM (Recency, Frequency, Monetary) analysis principles.The code performed the following feature engineering steps: New Engineered FeaturesNew FeatureFormulaE-commerce SignificanceAverage_Item_Price$\text{Total Spend} / \text{Items Purchased}$This captures the customer's value segment. Customers with a high average item price are likely high-end shoppers, which may strongly correlate with satisfaction and membership level.Engagement_Score$\text{Items Purchased} / \text{Days Since Last Purchase}$This is a measure of customer frequency/volume relative to their recency. A high score indicates a customer who buys a large number of items and does so frequently (low days since last purchase), suggesting high engagement.Recency_Value_Ratio$\text{Total Spend} / \text{Days Since Last Purchase}$This is a measure of the customer's monetary value relative to recency. A high ratio indicates a customer who spends a high amount and has purchased recently, suggesting a high-value, active customer.

New, highly useful features were engineered from the existing raw data, which often capture more meaningful customer behavior metrics than the original features alone. These new features are based on common e-commerce and RFM (Recency, Frequency, Monetary) analysis principles.

The code performed the following feature engineering steps:

New FeatureFormulaE-commerce SignificanceAverage_Item_Price$\text{Total Spend} / \text{Items Purchased}$This captures the customer's value segment. Customers with a high average item price are likely high-end shoppers, which may strongly correlate with satisfaction and membership level.Engagement_Score$\text{Items Purchased} / \text{Days Since Last Purchase}$This is a measure of customer frequency/volume relative to their recency. A high score indicates a customer who buys a large number of items and does so frequently (low days since last purchase), suggesting high engagement.Recency_Value_Ratio$\text{Total Spend} / \text{Days Since Last Purchase}$This is a measure of the customer's monetary value relative to recency. A high ratio indicates a customer who spends a high amount and has purchased recently, suggesting a high-value, active customer.

### Engineer new useful features
|New Feature	 | Formula | E-commerce Significance
|---|---|---|
|Average_Item_Price | {Total Spend} / {Items Purchased} | This captures the customer's value segment. Customers with a high average item price are likely high-end shoppers, which may strongly correlate with satisfaction and membership level.|

|Engagement_Score | {Items Purchased} / {Days Since Last Purchase} | This is a measure of customer frequency/volume relative to their recency. A high score indicates a customer who buys a large number of items and does so frequently (low days since last purchase), suggesting high engagement.|

|Recency_Value_Ratio | {Total Spend} / {Days Since Last Purchase} | This is a measure of the customer's monetary value relative to recency. A high ratio indicates a customer who spends a high amount and has purchased recently, suggesting a high-value, active customer.|




Deal with missing data (do not just drop rows or columns without justifying this).

Satisfaction Level	Missing Count : 2 (out of 350)

Strategy : Drop rows

Justification : Since Satisfaction Level is the target variable for classification, missing values must be handled carefully. Dropping the 2 rows with missing data is the most robust strategy because 2 rows represent only $0.57\%$ of the total data. Dropping them prevents the risk of introducing bias or inaccuracy that would come from trying to impute a categorical target variable with such low frequency.

The two rows with missing Satisfaction Level have been successfully dropped, resulting in a dataset with 348 rows and no missing data.

Create dummy variables if necessary.

In [75]:
# --- Feature Transformation Summary ---
#
# Original Feature     | Transformation        | New Column(s) Created         | Reason
# -----------------------------------------------------------------------------------------------------------------------------
# Gender               | One-Hot Encoding      | Gender_Male                   | Nominal (no order), avoids multicollinearity by using Female as the baseline.
# City                 | One-Hot Encoding      | City_Houston, City_Los Angeles, etc. | Nominal (no order), with Chicago being the baseline (when all city dummies are 0).
# Membership Type      | Ordinal Encoding      | Membership Type Encoded (1, 2, 3)    | Ordinal (clear order: Bronze < Silver < Gold).
# Satisfaction Level   | Label Encoding        | Satisfaction Level Encoded (0, 1, 2) | Target variable encoding for classification.

This explanation summarizes the complete data preprocessing workflow (covering feature selection, missing data handling, feature engineering, and transformation) performed on the E-commerce Customer Behavior Dataset to prepare it for a machine learning classification model (likely predicting Satisfaction Level).


The primary goal of this process was to ensure all data is in a numerical format, highly predictive, and free of issues like missing values or multicollinearity, adhering to the principle of avoiding data snooping by transforming features before model training.

Step 1: Feature Selection (Dropping Non-Useful Features)

In [81]:
# Feature: Customer ID | Action: Dropped | Justification: This is a unique, arbitrary identifier with no predictive value. Retaining it would lead to model overfitting and prevent generalization.

Step 2: Dealing with Missing DataInspection: An initial check revealed that only the Satisfaction Level column contained missing values (2 out of 350 rows).Action & Justification:Action: The 2 rows with missing Satisfaction Level were dropped.Justification: The Satisfaction Level is the target variable. Since 2 rows represent less than $1\%$ of the dataset, dropping them is the safest and most robust strategy. Imputing a categorical target variable risks introducing bias that could skew the model's learning process. The remaining 348 rows are clean.

Step 3: Feature Engineering (Creating Predictive Variables)
New features were engineered based on established e-commerce metrics (like RFM: Recency, Frequency, Monetary) to capture deeper insights into customer behavior.

In [85]:
# New Feature: Average_Item_Price | Formula: Total Spend / Items Purchased | Rationale: Measures customer value segment (budget vs. luxury).
# New Feature: Engagement_Score | Formula: Items Purchased / Days Since Last Purchase | Rationale: Measures customer activity/frequency. Higher score = more engaged.
# New Feature: Recency_Value_Ratio | Formula: Total Spend / Days Since Last Purchase | Rationale: Measures monetary value per day of inactivity, combining Recency and Monetary metrics.

Step 4: Feature Transformation and Encoding (Preparing for Modeling)
All remaining non-numerical features were converted into a numerical format suitable for machine learning algorithms.

"""
Feature Transformation and Encoding Summary:

| Original Feature | Action Taken | Rationale |
|:---|:---|:---|
| Discount Applied | Converted from Boolean to Integer | Mapped boolean values (True/False) to integers (1/0) for direct numerical model input. |
| Gender & City | One-Hot Encoding (Dummy Variables) | Nominal features (no intrinsic order) required binary columns. drop_first=True was used to prevent multicollinearity (Dummy Variable Trap). |
| Membership Type | Ordinal Encoding | Ordinal feature ($\text{Bronze} < \text{Silver} < \text{Gold}$) was mapped to integers ($0, 1, 2$) to preserve rank information. |
| Satisfaction Level | Label Encoding (Target) | Categorical classes ($\text{Unsatisfied}=0, \text{Neutral}=1, \text{Satisfied}=2$) were mapped to integers to prepare for a multi-class classification model. |
"""